<a href="https://colab.research.google.com/github/saraahmedi20002-source/git-test/blob/main/supervised_meta_blocking_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cloner le repo GitHub complet
!git clone https://github.com/Gaglia88/sparker.git

# Aller dans le dossier 'python' du projet
%cd sparker/python

!pip install pyspark findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession


Cloning into 'sparker'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 782 (delta 92), reused 121 (delta 41), pack-reused 585 (from 1)
Receiving objects: 100% (782/782), 49.54 MiB | 5.79 MiB/s, done.
Resolving deltas: 100% (378/378), done.
Updating files: 100% (208/208), done.
/content/sparker/python


In [3]:
import sys
sys.path.append('/content/sparker/python')

In [4]:
import sparker
from google.colab import files
uploaded = files.upload()
filename1 = list(uploaded.keys())[0]
from google.colab import files
uploaded = files.upload()
filename2 = list(uploaded.keys())[0]
from google.colab import files
uploaded = files.upload()
filename3 = list(uploaded.keys())[0]

/content/sparker/python/sparker/blockers.py:38: SyntaxWarning: invalid escape sequence '\W'
  split = list(map(lambda x: (x[0], re.split('\W+', x[1].lower())), valid_keys))
/content/sparker/python/sparker/blocking_strategies.py:26: SyntaxWarning: invalid escape sequence '\W'
  split = map(lambda x: (x[0], re.split('\W+', x[1])), values)
/content/sparker/python/sparker/blocking_strategies.py:51: SyntaxWarning: invalid escape sequence '\W'
  split = map(lambda x: re.split('\W+', x), values)
/content/sparker/python/sparker/attribute_clustering.py:135: SyntaxWarning: invalid escape sequence '\W'
  re.split('\W+', a.value.lower())))), attributes)


Saving dataset1.json to dataset1.json


Saving dataset2.json to dataset2.json


Saving groundtruth.json to groundtruth.json


In [5]:

import time
import pandas as pd

start_time = time.time()
profiles1 = sparker.JSONWrapper.load_profiles(filename1,
                                            real_id_field = "realProfileID",
                                              source_id=1)

# Profiles contained in the first dataset
#profiles1 = sparker.CSVWrapper.load_profiles(filename1, real_id_field = "custom_id", source_id=1)
df = pd.read_json(filename1,lines=True)
print("Nombre d’éléments d1:", len(df))
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
separator_ids = [separator_id]


profiles2 = sparker.JSONWrapper.load_profiles(filename2,
                                             start_id_from = separator_id+1,
                                              real_id_field = "realProfileID",
                                             source_id=2)
#profiles2 = sparker.CSVWrapper.load_profiles(filename2, real_id_field = "custom_id", source_id=1)
df = pd.read_json(filename2,lines=True)
print("Nombre d’éléments d2:", len(df))
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()
profiles = profiles1.union(profiles2)

gt = sparker.JSONWrapper.load_groundtruth(filename3, 'id1', 'id2')
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

Nombre d’éléments d1: 2516
Nombre d’éléments d2: 61353


In [6]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
print(blocks)

PythonRDD[21] at RDD at PythonRDD.scala:53


In [7]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.\
                                                                            block_filtering_quick(blocks_purged,
                                                                                                  0.8,
                                                                                                  separator_ids)

In [8]:
features = sparker.FeatureGenerator.generate_features(profiles, blocks_after_filtering, separator_ids, new_gt, False)

In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, BooleanType

In [10]:
# Features to employ
features_set = ["cfibf", "raccb", "js", "rs", "aejs", "nrs", "wjs"]
 #"numCompP1", "numCompP2",
va = VectorAssembler(inputCols=features_set, outputCol="features")

df = va.transform(features)

In [11]:
# Number of samples per class (actually spark do not ensure this exact value during sampling)
n_samples = 20

# Sampling of matching pairs
matches = df.where("is_match == 1")
print("dddddddddddd")
non_matches = df.where("is_match == 0")
print(df.collect)
m_n = n_samples/matches.count()
m_train, m_test = matches.randomSplit([m_n, 1-m_n])

# Sampling of non-matching pairs
non_matches = df.where("is_match == 0")
print(non_matches.count())
nm_n = n_samples/non_matches.count()
nm_train, nm_test = non_matches.randomSplit([nm_n, 1-nm_n])

# Train/Test
train = m_train.union(nm_train)
test = m_test.union(nm_test)
end_time = time.time()
time1=end_time - start_time

dddddddddddd
<bound method DataFrame.collect of DataFrame[p1: int, p2: int, cfibf: float, raccb: float, js: float, numCompP1: bigint, numCompP2: bigint, rs: float, aejs: float, nrs: float, wjs: float, is_match: int, features: vector]>
2586547


In [12]:
lr = LogisticRegression(featuresCol='features',
                        labelCol='is_match',
                        predictionCol='prediction',
                        maxIter=1000,
                        probabilityCol='probability'
                        )

#print(f"⏱ Temps écoulé : {end_time - start_time:.2f} secondes")
# Training
model = lr.fit(train)
# Performs the predictions
#start_time = time.time()
predictions = model.transform(test)

# Get the results as the probability of each pair (edge) of being a match
get_p_match = f.udf(lambda v: float(v[1]), FloatType())
edges = predictions\
        .withColumn("p_match", get_p_match("probability"))\
        .select("p1", "p2", "p_match", "is_match")

edges.cache()
edges.count()


2588811

In [13]:
pruned_edges = sparker.SupervisedMB.rcnp(edges,profiles,blocks)
pc, pq, f1 = sparker.SupervisedMB.get_stats(pruned_edges, new_gt)
print("Recall "+str(pc))
print("Precision "+str(pq))
print("F1 "+str(f1))

end_time = time.time()
time2=end_time - start_time
print(f"⏱ Temps écoulé : {time2:.2f} secondes")

Recall 0.9480069324090121
Precision 0.18348008385744236
F1 0.30745450713131456
⏱ Temps écoulé : 1368.05 secondes
